This document discusses automatically generating presentations from notebooks.  The __reveal.js__ view of a notebook provides an alternative interface to view as slides and in a sorted layout view.  

# Interpretting notebooks as presentations.

In [8]:
    import IPython; get_ipython = IPython.get_ipython
    ip = get_ipython()
    if ip:
        %reload_ext pidgin

IndentationError: expected an indented block (<ipython-input-8-29feb61f8d8c>, line 4)

In [19]:
## Converting a notebook to a presentation.

The `nbconvert.SlidesExporter` exports `nbformat` documents as presentations;
`assert issubclass(nbconvert.SlidesExporter, nbconvert.HTMLExporter)`.
    
    import nbconvert, nbformat, traitlets

'## Converting a notebook to a presentation.\n\nThe `nbconvert.SlidesExporter` exports `nbformat` documents as presentations;\n`assert issubclass(nbconvert.SlidesExporter, nbconvert.HTMLExporter)`.\n    \n    import nbconvert, nbformat, traitlets'

In [20]:
### Slide show metadata.

`nbconvert.SlidesExporter` uses cell metadata to create `"slide subslide fragment"` `"slide_type"`s.

    def add_slide_metadata(cell):
`add_slide_metadata` adds `"slideshow"` data to a __cell__ using the headers and horizontal rules as delimiters.

        source = ''.join(cell['source'])
        type = F"- slide slide subslide fragment fragment fragment".split(
        )[max(get_levels(source) or [0])]
`"slideshow"` is a keyword specific to the `nbconvert.SlidesExporter`, we only update the metadata 
if the keywords don't exist.

        cell.metadata['slideshow'] = cell.metadata.get('slideshow', {})
        cell.metadata['slideshow'].update(slide_type=cell.metadata['slideshow'].get(
            'slide_type', type
        ))
        return cell

'### Slide show metadata.\n\n`nbconvert.SlidesExporter` uses cell metadata to create `"slide subslide fragment"` `"slide_type"`s.\n\n    def add_slide_metadata(cell):\n`add_slide_metadata` adds `"slideshow"` data to a __cell__ using the headers and horizontal rules as delimiters.\n\n        source = \'\'.join(cell[\'source\'])\n        type = F"- slide slide subslide fragment fragment fragment".split(\n        )[max(get_levels(source) or [0])]\n`"slideshow"` is a keyword specific to the `nbconvert.SlidesExporter`, we only update the metadata \nif the keywords don\'t exist.\n\n        cell.metadata[\'slideshow\'] = cell.metadata.get(\'slideshow\', {})\n        cell.metadata[\'slideshow\'].update(slide_type=cell.metadata[\'slideshow\'].get(\n            \'slide_type\', type\n        ))\n        return cell'

In [21]:
### Extracting a table of contents with `mistune`

    import mistune
    class HeadingRenderer(mistune.Renderer):
        levels = None
        def header(self, text, level, raw=None):
            self.levels = (self.levels or set())
            self.levels.add(level)
            return text

'### Extracting a table of contents with `mistune`\n\n    import mistune\n    class HeadingRenderer(mistune.Renderer):\n        levels = None\n        def header(self, text, level, raw=None):\n            self.levels = (self.levels or set())\n            self.levels.add(level)\n            return text'

In [22]:
    def get_levels(text):
```
>>> assert get_levels("# asdf\\n## asdfadf") == {1, 2}
```

        renderer = HeadingRenderer()
        mistune.Markdown(renderer=renderer).render(text)

        return renderer.levels

'    def get_levels(text):\n```\n>>> assert get_levels("# asdf\\\\n## asdfadf") == {1, 2}\n```\n\n        renderer = HeadingRenderer()\n        mistune.Markdown(renderer=renderer).render(text)\n\n        return renderer.levels'

In [23]:
### Adding the metadata

Take image of the metadata.

    def add_slide_data(cell):
        source = ''.join(cell['source'])
        type = F"subslide slide slide subslide fragment fragment fragment".split(
        )[max(get_levels(source) or [0])]
        cell.metadata['slideshow'] = cell.metadata.get('slideshow', {})
        cell.metadata['slideshow'].update(slide_type=cell.metadata['slideshow'].get('slide_type', type))
        return cell

'### Adding the metadata\n\nTake image of the metadata.\n\n    def add_slide_data(cell):\n        source = \'\'.join(cell[\'source\'])\n        type = F"subslide slide slide subslide fragment fragment fragment".split(\n        )[max(get_levels(source) or [0])]\n        cell.metadata[\'slideshow\'] = cell.metadata.get(\'slideshow\', {})\n        cell.metadata[\'slideshow\'].update(slide_type=cell.metadata[\'slideshow\'].get(\'slide_type\', type))\n        return cell'

In [24]:
### An `nbconvert.preprocessors.Preprocessor`
    
    class AutomaticallyPresent(nbconvert.preprocessors.Preprocessor):
```
>>> assert nbconvert.SlidesExporter(preprocessors=[AutomaticallyPresent()]).from_filename(
...    globals().get('__file__', 'Untitled104.ipynb'))[0]
```

        def preprocess_cell(self, cell, resources, index):
            if 'source' not in cell: return cell, resources
            
            if cell['cell_type'] == 'code':
                self.is_pidgin = self.is_pidgin or (
                    'load_ext pidgin' in ''.join(cell['source']))
               
            if cell['cell_type'] in (['markdown'] + (
                self.is_pidgin and ['code'] or []
            )):
                if (cell['cell_type'] == 'markdown') or ('outputs' in cell) and cell['outputs']:
                    cell = add_slide_data(cell)
                
            return cell, resources
        
        def preprocess(self, nb, resources):
            self.last_heading = None
            self.is_pidgin = False
            nb, resources = super().preprocess(nb, resources)
            return nbconvert.exporters.slides.prepare(nb), resources


"### An `nbconvert.preprocessors.Preprocessor`\n    \n    class AutomaticallyPresent(nbconvert.preprocessors.Preprocessor):\n```\n>>> assert nbconvert.SlidesExporter(preprocessors=[AutomaticallyPresent()]).from_filename(\n...    globals().get('__file__', 'Untitled104.ipynb'))[0]\n```\n\n        def preprocess_cell(self, cell, resources, index):\n            if 'source' not in cell: return cell, resources\n            \n            if cell['cell_type'] == 'code':\n                self.is_pidgin = self.is_pidgin or (\n                    'load_ext pidgin' in ''.join(cell['source']))\n               \n            if cell['cell_type'] in (['markdown'] + (\n                self.is_pidgin and ['code'] or []\n            )):\n                if (cell['cell_type'] == 'markdown') or ('outputs' in cell) and cell['outputs']:\n                    cell = add_slide_data(cell)\n                \n            return cell, resources\n        \n        def preprocess(self, nb, resources):\n            self.last_heading = None\n            self.is_pidgin = False\n            nb, resources = super().preprocess(nb, resources)\n            return nbconvert.exporters.slides.prepare(nb), resources\n"

## Automatically adding the metadata.

In [42]:
    def _repr_html_(include=None, exclude=None):
        import vdom.svg
        exporter = nbconvert.SlidesExporter(
            preprocessors=[AutomaticallyPresent()],
            reveal_scroll=True,
            exclude_input=True,
        )
        return vdom.svg.iframe(
            srcdoc=exporter.from_filename('Untitled104.ipynb')[0],
            width="100%", height="500px",
            **{"class": "pdg-Frame"}
        )._repr_html_()

In [43]:
    import pygments

    __import__(__name__)